# Storing Fantasy Premier League data in a database - part 2
- Author: Steffan Rees
- 19/04/2022

## Background
This notebook follows on from **Storing Fantasy Premier League data in a database - part 1** where we extracted fantasy premier league data from the API and stored the transformed data in a SQLite database.

In this notebook we will query the data using SQL and pandas.

It's worth noting that the database and its tables can also be accessed using SQLite Studio.

Pandas has a guide for comparing pandas operations with SQL https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_sql.html.

In [1]:
# Import libraries
import pandas as pd
import sqlite3
from pandasql import sqldf

In [2]:
# Connect to the database
def sql_connection(db):
    try:
        con = sqlite3.connect(db)
    except Error:
        print(Error)
        
    cursor = con.cursor()
    return con, cursor

In [3]:
con, cursor = sql_connection('fpl.db')

## List the tables in the database

In [4]:
# Using SQLite
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('gameweeks',), ('teams',), ('positions',), ('players',), ('player_stats',), ('fixtures',)]


In [5]:
# Using pandas
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", con)

,name
0,gameweeks
1,teams
2,positions
3,players
4,player_stats
5,fixtures


## Import data into dataframe

### Teams

In [6]:
teams = pd.read_sql_query("SELECT * FROM teams", con)
print(teams.shape)
teams.head()

(20, 21)


,team_id,team_code,team_name,short_name,draw,form,loss,played,points,position,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,1,3,Arsenal,ARS,0,None,0,0,0,0,...,None,0,0,1250,1270,1150,1210,1190,1220,1
1,2,7,Aston Villa,AVL,0,None,0,0,0,0,...,None,0,0,1100,1100,1140,1110,1090,1090,2
2,3,94,Brentford,BRE,0,None,0,0,0,0,...,None,0,0,1060,1070,1120,1150,1080,1120,130
3,4,36,Brighton,BHA,0,None,0,0,0,0,...,None,0,0,1100,1090,1160,1160,1100,1120,131
4,5,90,Burnley,BUR,0,None,0,0,0,0,...,None,0,0,1060,1060,1080,1130,1060,1100,43


In [7]:
teams.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   team_id                20 non-null     int64 
 1   team_code              20 non-null     int64 
 2   team_name              20 non-null     object
 3   short_name             20 non-null     object
 4   draw                   20 non-null     int64 
 5   form                   0 non-null      object
 6   loss                   20 non-null     int64 
 7   played                 20 non-null     int64 
 8   points                 20 non-null     int64 
 9   position               20 non-null     int64 
 10  strength               20 non-null     int64 
 11  team_division          0 non-null      object
 12  unavailable            20 non-null     int64 
 13  win                    20 non-null     int64 
 14  strength_overall_home  20 non-null     int64 
 15  strength_overall_away  20

### Fixtures

In [8]:
fixtures = pd.read_sql_query("SELECT * FROM fixtures", con, dtype={'gameweek': 'Int8',
                                                                   'gameweek': 'Int8',
                                                                   'started': 'Int8',
                                                                   'team_a_score': 'Int8',
                                                                   'team_h_score': 'Int8', 
                                                                  })
print(fixtures.shape)
fixtures.head()

(380, 16)


,fixture_id,fixture_code,gameweek,finished,finished_provisional,kickoff_time,minutes,provisional_start_time,started,away_team_id,team_a_score,home_team_id,team_h_score,team_h_difficulty,team_a_difficulty,pulse_id
0,1,2210271,1,1,1,2021-08-13T19:00:00Z,90,0,1,1,0,3,2,4,2,66342
1,2,2210272,1,1,1,2021-08-14T14:00:00Z,90,0,1,4,2,5,1,2,2,66343
2,3,2210273,1,1,1,2021-08-14T14:00:00Z,90,0,1,7,0,6,3,2,4,66344
3,4,2210274,1,1,1,2021-08-14T14:00:00Z,90,0,1,16,1,8,3,2,2,66345
4,5,2210275,1,1,1,2021-08-14T14:00:00Z,90,0,1,20,0,9,1,3,3,66346


In [9]:
fixtures.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   fixture_id              380 non-null    int64 
 1   fixture_code            380 non-null    int64 
 2   gameweek                380 non-null    Int8  
 3   finished                380 non-null    int64 
 4   finished_provisional    380 non-null    int64 
 5   kickoff_time            380 non-null    object
 6   minutes                 380 non-null    int64 
 7   provisional_start_time  380 non-null    int64 
 8   started                 380 non-null    Int8  
 9   away_team_id            380 non-null    int64 
 10  team_a_score            341 non-null    Int8  
 11  home_team_id            380 non-null    int64 
 12  team_h_score            341 non-null    Int8  
 13  team_h_difficulty       380 non-null    int64 
 14  team_a_difficulty       380 non-null    int64 
 15  pulse_

In [10]:
fixtures.started.describe()

count    380.000000
mean       0.897368
std        0.303877
min        0.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: started, dtype: float64

### Players

In [11]:
players = pd.read_sql_query("SELECT * FROM players", con)
print(players.shape)
players.head()

(730, 66)


,player_id,player_code,first_name,second_name,web_name,team_id,team_code,chance_of_playing_next_round,chance_of_playing_this_round,cost_change_event,...,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,1,80201,Bernd,Leno,Leno,1,3,100.0,100.0,0,...,594,60,438,30,NaN,,NaN,,NaN,
1,2,115918,Rúnar Alex,Rúnarsson,Rúnarsson,1,3,0.0,0.0,0,...,485,19,538,54,NaN,,NaN,,NaN,
2,3,47431,Willian,Borges Da Silva,Willian,1,3,0.0,0.0,0,...,726,303,726,303,NaN,,NaN,,NaN,
3,4,54694,Pierre-Emerick,Aubameyang,Aubameyang,1,3,0.0,0.0,0,...,51,20,154,29,NaN,,NaN,,NaN,
4,5,58822,Cédric,Soares,Cédric,1,3,100.0,100.0,0,...,319,104,269,89,2.0,,3.0,,NaN,


In [12]:
players.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 66 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   player_id                             730 non-null    int64  
 1   player_code                           730 non-null    int64  
 2   first_name                            730 non-null    object 
 3   second_name                           730 non-null    object 
 4   web_name                              730 non-null    object 
 5   team_id                               730 non-null    int64  
 6   team_code                             730 non-null    int64  
 7   chance_of_playing_next_round          565 non-null    float64
 8   chance_of_playing_this_round          565 non-null    float64
 9   cost_change_event                     730 non-null    int64  
 10  cost_change_event_fall                730 non-null    int64  
 11  cost_change_start  

### Positions

In [13]:
positions = pd.read_sql_query("SELECT * FROM positions", con)
print(positions.shape)
positions.head()

(4, 9)


,position_id,position_name_short,position_name,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,GKP,Goalkeeper,2,1,1,1,12.0,83
1,2,DEF,Defender,5,3,5,0,NaN,247
2,3,MID,Midfielder,5,2,5,0,NaN,305
3,4,FWD,Forward,3,1,3,0,NaN,95


In [14]:
positions.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   position_id           4 non-null      int64  
 1   position_name_short   4 non-null      object 
 2   position_name         4 non-null      object 
 3   squad_select          4 non-null      int64  
 4   squad_min_play        4 non-null      int64  
 5   squad_max_play        4 non-null      int64  
 6   ui_shirt_specific     4 non-null      int64  
 7   sub_positions_locked  1 non-null      float64
 8   element_count         4 non-null      int64  
dtypes: float64(1), int64(6), object(2)
memory usage: 416.0+ bytes


## Queries

### Add team name and position to players data

#### SQLite

In [15]:
query = """
SELECT 
    pl.player_id,
    pl.first_name,
    pl.second_name,
    pl.team_id,
    t.team_name,
    pl.position_id,
    po.position_name
FROM players as pl 
INNER JOIN positions as po
    ON po.position_id = pl.position_id
INNER JOIN teams as t
    on t.team_id = pl.team_id
ORDER BY pl.player_id ASC;
"""

cursor.execute(query)
print(cursor.fetchall())

[(1, 'Bernd', 'Leno', 1, 'Arsenal', 1, 'Goalkeeper'), (2, 'Rúnar Alex', 'Rúnarsson', 1, 'Arsenal', 1, 'Goalkeeper'), (3, 'Willian', 'Borges Da Silva', 1, 'Arsenal', 3, 'Midfielder'), (4, 'Pierre-Emerick', 'Aubameyang', 1, 'Arsenal', 4, 'Forward'), (5, 'Cédric', 'Soares', 1, 'Arsenal', 2, 'Defender'), (6, 'Alexandre', 'Lacazette', 1, 'Arsenal', 4, 'Forward'), (7, 'Granit', 'Xhaka', 1, 'Arsenal', 3, 'Midfielder'), (8, 'Pablo', 'Marí', 1, 'Arsenal', 2, 'Defender'), (9, 'Héctor', 'Bellerín', 1, 'Arsenal', 2, 'Defender'), (10, 'Calum', 'Chambers', 2, 'Aston Villa', 2, 'Defender'), (11, 'Sead', 'Kolasinac', 1, 'Arsenal', 2, 'Defender'), (12, 'Mohamed Naser', 'El Sayed Elneny', 1, 'Arsenal', 3, 'Midfielder'), (13, 'Ainsley', 'Maitland-Niles', 1, 'Arsenal', 3, 'Midfielder'), (14, 'Rob', 'Holding', 1, 'Arsenal', 2, 'Defender'), (15, 'Thomas', 'Partey', 1, 'Arsenal', 3, 'Midfielder'), (16, 'Kieran', 'Tierney', 1, 'Arsenal', 2, 'Defender'), (17, 'Nicolas', 'Pépé', 1, 'Arsenal', 3, 'Midfielder'), 

In [17]:
# Storing the query results in a dataframe
query = """
SELECT 
    pl.player_id,
    pl.first_name,
    pl.second_name,
    pl.team_id,
    t.team_name,
    pl.position_id,
    po.position_name
FROM players as pl 
INNER JOIN positions as po
    ON po.position_id = pl.position_id
INNER JOIN teams as t
    on t.team_id = pl.team_id
ORDER BY pl.player_id ASC;
"""

pd.read_sql_query(query, con)

,player_id,first_name,second_name,team_id,team_name,position_id,position_name
0,1,Bernd,Leno,1,Arsenal,1,Goalkeeper
1,2,Rúnar Alex,Rúnarsson,1,Arsenal,1,Goalkeeper
2,3,Willian,Borges Da Silva,1,Arsenal,3,Midfielder
3,4,Pierre-Emerick,Aubameyang,1,Arsenal,4,Forward
4,5,Cédric,Soares,1,Arsenal,2,Defender
...,...,...,...,...,...,...,...
725,726,Dara,Costelloe,5,Burnley,3,Midfielder
726,727,Shola,Shoretire,13,Man Utd,3,Midfielder
727,728,Álvaro,Fernández,13,Man Utd,2,Defender
728,729,Matthew,Craig,17,Spurs,3,Midfielder


#### Pandas

In [19]:
(players[['player_id',
          'first_name',
          'second_name',
          'team_id',
          'position_id'
         ]]
 .assign(team_name = players.team_id.map(teams.set_index('team_id').team_name),
         position_name = players.position_id.map(positions.set_index('position_id').position_name)
        )
 .reindex(columns=['player_id',
                   'first_name',
                   'second_name',
                   'team_id',
                   'team_name',
                   'position_id',
                   'position_name'
                  ])
)

,player_id,first_name,second_name,team_id,team_name,position_id,position_name
0,1,Bernd,Leno,1,Arsenal,1,Goalkeeper
1,2,Rúnar Alex,Rúnarsson,1,Arsenal,1,Goalkeeper
2,3,Willian,Borges Da Silva,1,Arsenal,3,Midfielder
3,4,Pierre-Emerick,Aubameyang,1,Arsenal,4,Forward
4,5,Cédric,Soares,1,Arsenal,2,Defender
...,...,...,...,...,...,...,...
725,726,Dara,Costelloe,5,Burnley,3,Midfielder
726,727,Shola,Shoretire,13,Man Utd,3,Midfielder
727,728,Álvaro,Fernández,13,Man Utd,2,Defender
728,729,Matthew,Craig,17,Spurs,3,Midfielder


In [20]:
# Using pandasql
pysqldf = lambda q: sqldf(q, globals())
pysqldf("""
SELECT 
    pl.player_id,
    pl.first_name,
    pl.second_name,
    pl.team_id,
    t.team_name,
    pl.position_id,
    po.position_name
FROM players as pl 
INNER JOIN positions as po
    ON po.position_id = pl.position_id
INNER JOIN teams as t
    on t.team_id = pl.team_id
ORDER BY pl.player_id ASC;
"""
)

,player_id,first_name,second_name,team_id,team_name,position_id,position_name
0,1,Bernd,Leno,1,Arsenal,1,Goalkeeper
1,2,Rúnar Alex,Rúnarsson,1,Arsenal,1,Goalkeeper
2,3,Willian,Borges Da Silva,1,Arsenal,3,Midfielder
3,4,Pierre-Emerick,Aubameyang,1,Arsenal,4,Forward
4,5,Cédric,Soares,1,Arsenal,2,Defender
...,...,...,...,...,...,...,...
725,726,Dara,Costelloe,5,Burnley,3,Midfielder
726,727,Shola,Shoretire,13,Man Utd,3,Midfielder
727,728,Álvaro,Fernández,13,Man Utd,2,Defender
728,729,Matthew,Craig,17,Spurs,3,Midfielder


### Count of fixtures per gameweek for each team

#### SQL

In [21]:
query = """
SELECT
    t2.gameweek,
    t2.team_id,
    t.team_name,
    COUNT(*) AS Fixtures
FROM
(
SELECT
    t1.gameweek,
    t1.team_id
FROM
(
    SELECT 
       f1.gameweek,
       f1.home_team_id AS team_id
    FROM fixtures as f1
    UNION ALL
    SELECT 
       f2.gameweek,
       f2.away_team_id AS team_id
    FROM fixtures as f2
) AS t1
) AS t2
INNER JOIN teams as t
    ON t.team_id = t2.team_id
WHERE t2.gameweek IS NOT NULL
GROUP BY 
    t2.gameweek,
    t2.team_id,
    t.team_name
ORDER BY 
    t2.gameweek ASC,
    t.team_name ASC
"""

cursor.execute(query)
print(cursor.fetchall())

[(1, 1, 'Arsenal', 1), (1, 2, 'Aston Villa', 1), (1, 3, 'Brentford', 1), (1, 4, 'Brighton', 1), (1, 5, 'Burnley', 1), (1, 6, 'Chelsea', 1), (1, 7, 'Crystal Palace', 1), (1, 8, 'Everton', 1), (1, 10, 'Leeds', 1), (1, 9, 'Leicester', 1), (1, 11, 'Liverpool', 1), (1, 12, 'Man City', 1), (1, 13, 'Man Utd', 1), (1, 14, 'Newcastle', 1), (1, 15, 'Norwich', 1), (1, 16, 'Southampton', 1), (1, 17, 'Spurs', 1), (1, 18, 'Watford', 1), (1, 19, 'West Ham', 1), (1, 20, 'Wolves', 1), (2, 1, 'Arsenal', 1), (2, 2, 'Aston Villa', 1), (2, 3, 'Brentford', 1), (2, 4, 'Brighton', 1), (2, 5, 'Burnley', 1), (2, 6, 'Chelsea', 1), (2, 7, 'Crystal Palace', 1), (2, 8, 'Everton', 1), (2, 10, 'Leeds', 1), (2, 9, 'Leicester', 1), (2, 11, 'Liverpool', 1), (2, 12, 'Man City', 1), (2, 13, 'Man Utd', 1), (2, 14, 'Newcastle', 1), (2, 15, 'Norwich', 1), (2, 16, 'Southampton', 1), (2, 17, 'Spurs', 1), (2, 18, 'Watford', 1), (2, 19, 'West Ham', 1), (2, 20, 'Wolves', 1), (3, 1, 'Arsenal', 1), (3, 2, 'Aston Villa', 1), (3, 3, 

#### Pandas

In [22]:
(fixtures[['gameweek', 'home_team_id', 'away_team_id']]
 .append(fixtures.rename(columns={**dict(zip(['home_team_id'], ['away_team_id'])), **dict(zip(['away_team_id'], ['home_team_id']))}))
 .sort_index(ignore_index=True)
 .drop(columns=['away_team_id'])
 .rename(columns={'home_team_id': 'team_id'})
 .groupby(['gameweek', 'team_id']).size().to_frame('fixtures').reset_index()
 .assign(team_name = lambda x: x.team_id.map(teams.set_index('team_id').team_name))
 .astype({'gameweek': 'int8'})
 .reindex(columns=['gameweek',
                   'team_id',
                   'team_name',
                   'fixtures'
                  ])
)

,gameweek,team_id,team_name,fixtures
0,1,1,Arsenal,1
1,1,2,Aston Villa,1
2,1,3,Brentford,1
3,1,4,Brighton,1
4,1,5,Burnley,1
...,...,...,...,...
694,38,16,Southampton,1
695,38,17,Spurs,1
696,38,18,Watford,1
697,38,19,West Ham,1


In [23]:
# Using pandasql
pysqldf = lambda q: sqldf(q, globals())
pysqldf("""
SELECT
    t2.gameweek,
    t2.team_id,
    t.team_name,
    COUNT(*) AS Fixtures
FROM
(
SELECT
    t1.gameweek,
    t1.team_id
FROM
(
    SELECT 
       f1.gameweek,
       f1.home_team_id AS team_id
    FROM fixtures as f1
    UNION ALL
    SELECT 
       f2.gameweek,
       f2.away_team_id AS team_id
    FROM fixtures as f2
) AS t1
) AS t2
INNER JOIN teams as t
    ON t.team_id = t2.team_id
WHERE t2.gameweek IS NOT NULL
GROUP BY 
    t2.gameweek,
    t2.team_id,
    t.team_name
ORDER BY 
    t2.gameweek ASC,
    t.team_name ASC
"""
)

,gameweek,team_id,team_name,Fixtures
0,1,1,Arsenal,1
1,1,2,Aston Villa,1
2,1,3,Brentford,1
3,1,4,Brighton,1
4,1,5,Burnley,1
...,...,...,...,...
694,38,16,Southampton,1
695,38,17,Spurs,1
696,38,18,Watford,1
697,38,19,West Ham,1


### Close connection to the database

In [24]:
con.close()

## Summary
We have explored some simple queries of the fpl database and compared SQL and pandas query equivalents.

Further queries may be added in the future.